In [ ]:
import pandas as pd
import os

In [ ]:
nyse_holidays = [
    "2023-01-02",
    "2023-01-16",
    "2023-02-20",
    "2023-04-14",
    "2023-05-29",
    "2023-07-04",
    "2023-09-04",
    "2023-11-23",
    "2023-12-25"
]
EXAMPLE = "AAPL"
STOCK_OPENING_HOUR = 10 #UTC
STOCK_OPENING_MIN = 30 #UTC
STOCK_CLOSE_HOUR = 20 #UTC
STOCK_CLOSE_MIN = 0 #UTC
news_path = os.path.join("./", "news_sentiments.csv")
stocks_path = os.path.join("./", "stocks.csv")
stocks = pd.read_csv(stocks_path)
news = pd.read_csv(news_path)

In [ ]:
# Make valid dates without timezone
news["date"] = pd.to_datetime(news['pub_date'],errors='coerce',utc=True).dt.tz_localize(None)
stocks['Date'] = pd.to_datetime(stocks["Date"], errors="coerce",utc=True).dt.tz_localize(None)
stocks.set_index('Date', inplace=True)

In [ ]:
def get_affected_day(date):
    close_hour = date.replace(hour=STOCK_CLOSE_HOUR, minute=STOCK_CLOSE_MIN)
    if (date > close_hour) or (date.strftime("%Y-%m-%d") in nyse_holidays):
        date = date.replace(hour=STOCK_OPENING_HOUR, minute=STOCK_OPENING_MIN)
        date = date + pd.Timedelta(days=1)
    return date

news["affected_day"] = news["date"].apply(get_affected_day)
news.tail()

In [ ]:
def add_stock_results(row):
    date = row["affected_day"].strftime("%Y-%m-%d")
    stock_values = stocks[
        (stocks['Ticker'] == row['ticker'])
    ]
    stock_values = stock_values.loc[date, ["Open","Close"]]
    if len(stock_values.values) != 0:
        stock_values = stock_values.values[0].tolist()
        stock_values.append(stock_values[0]-stock_values[1])
    else:
        stock_values = [None, None, None]
    return stock_values
    
news[
    ['Open', 'Close', 'Open-Close']
] = news.apply(lambda row: pd.Series(add_stock_results(row)), axis=1)

news.tail()

In [ ]:
news.to_csv("complete_data.csv", index=False)

In [ ]:
del news['title']
del news['description']
del news['pub_date']
del news['link']
del news['date']

In [ ]:
news.to_csv("trainment.csv", index=False)